In [1]:
import Utils
from DataLoader import DataLoader
from FeatureCreator import FeatureCreator
from Model import Model
from CouponCreator import CouponCreator

In [2]:
config = Utils.read_json('../config.json')
Utils.print_json(config)

{
    "data": {
        "path": "../data/",
        "files": {
            "baskets": "baskets.parquet",
            "coupons": "coupons.parquet",
            "coupons_index": "coupon_index.parquet"
        }
    },
    "model": {
        "test_week": 90,
        "n_shoppers": 2000,
        "n_products": 250,
        "train_window": 4,
        "trend_windows": [
            1,
            3,
            5
        ],
        "n_coupons": 5,
        "discounts": [
            0.15,
            0.2,
            0.25,
            0.3
        ]
    }
}


In [3]:
config['model']['test_week'] = 89

In [4]:
# DATA LOADING
dataloader = DataLoader(config)
dataset = dataloader.get_dataset()

Read dataset.parquet.gzip from cache...
dataset.parquet.gzip was not found in cache.
Executing callback function "create_dataset" to create dataset.parquet.gzip
57.5% memory reduction for baskets.parquet (from 2626.10 MB to 1116.09 MB)
60.0% memory reduction for coupons.parquet (from 1716.61 MB to 686.65 MB)
62.5% memory reduction for coupon_index.parquet (from 0.31 MB to 0.11 MB)
replaced missing prices with mode
Wrote dataset.parquet.gzip to ../cache/
Successfully read dataset.parquet.gzip into memory.


In [5]:
dataset

,week,shopper,product,price,discount,purchased,week_hist,elast,redemption_rate,costumer_redemption_rate,discount_buy
0,85,0,0,688.0,0.0,0.0,NaN,5.647909,0.039216,NaN,NaN
1,85,0,1,560.0,0.0,0.0,NaN,1.439524,0.104438,0.000000,1.000000
2,85,0,2,773.0,0.0,0.0,NaN,9.196110,0.037890,0.000000,1.000000
3,85,0,3,722.0,0.0,0.0,NaN,10.764508,0.020378,0.000000,1.000000
4,85,0,4,620.0,0.0,0.0,"[24, 40, 45, 51, 54, 58, 60, 70]",6.075778,0.057287,0.000000,0.272727
...,...,...,...,...,...,...,...,...,...,...,...
2499995,89,1999,245,549.0,0.0,1.0,"[5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70, 89]",3.938279,0.032201,0.666667,0.250000
2499996,89,1999,246,702.0,0.3,0.0,NaN,4.741070,0.044372,0.000000,1.000000
2499997,89,1999,247,670.0,0.0,0.0,NaN,4.759275,0.017694,0.000000,1.000000
2499998,89,1999,248,490.0,0.0,0.0,NaN,2.050100,0.127890,0.000000,1.000000


In [6]:
# FEATURE CREATION
feature_creator = FeatureCreator(dataset, config)
model_data = feature_creator.get_model_data()
model_data

Read model_data.parquet.gzip from cache...
model_data.parquet.gzip was not found in cache.
Executing callback function "create_features" to create model_data.parquet.gzip
Wrote model_data.parquet.gzip to ../cache/
Successfully read model_data.parquet.gzip into memory.


,week,shopper,product,price,discount,purchased,elast,redemption_rate,costumer_redemption_rate,discount_buy,product_cat,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq
0,85,0,0,688.0,0.0,0.0,5.647909,0.039216,NaN,NaN,0,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000
1,85,0,1,560.0,0.0,0.0,1.439524,0.104438,0.000000,1.000000,0,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000
2,85,0,2,773.0,0.0,0.0,9.196110,0.037890,0.000000,1.000000,0,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000
3,85,0,3,722.0,0.0,0.0,10.764508,0.020378,0.000000,1.000000,0,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000
4,85,0,4,620.0,0.0,0.0,6.075778,0.057287,0.000000,0.272727,0,0.0,0,1,0,15.0,0.0,0.0,0.0,0.094118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499995,89,1999,245,549.0,0.0,1.0,3.938279,0.032201,0.666667,0.250000,24,0.3,0,0,1,19.0,0.0,0.0,0.0,0.123596
2499996,89,1999,246,702.0,0.3,0.0,4.741070,0.044372,0.000000,1.000000,24,0.0,0,0,1,103.0,0.0,0.0,0.0,0.000000
2499997,89,1999,247,670.0,0.0,0.0,4.759275,0.017694,0.000000,1.000000,24,0.3,0,0,1,103.0,0.0,0.0,0.0,0.000000
2499998,89,1999,248,490.0,0.0,0.0,2.050100,0.127890,0.000000,1.000000,24,0.3,0,0,1,103.0,0.0,0.0,0.0,0.000000


In [8]:
# MODELING - Train-Test-Split
model = Model(model_data)
X_train, y_train, X_test, y_test = model.train_test_split(config)

/Users/leonhagel/Documents/coding/github/mlim-g2/env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/leonhagel/Documents/coding/github/mlim-g2/env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [9]:
X_train

,price,discount,elast,redemption_rate,costumer_redemption_rate,discount_buy,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_WOE,product_WOE
0,688.0,0.0,5.647909,0.039216,NaN,NaN,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-0.541070
1,560.0,0.0,1.439524,0.104438,0.000000,1.000000,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,0.638078
2,773.0,0.0,9.196110,0.037890,0.000000,1.000000,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-0.776683
3,722.0,0.0,10.764508,0.020378,0.000000,1.000000,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-1.401178
4,620.0,0.0,6.075778,0.057287,0.000000,0.272727,0.0,0,1,0,15.0,0.0,0.0,0.0,0.094118,0.039796,-0.128700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,549.0,0.0,0.656368,0.032201,0.666667,0.250000,0.0,0,0,1,18.0,0.0,0.0,0.0,0.125000,0.071599,-0.878618
1999996,702.0,0.0,4.783256,0.044372,0.000000,1.000000,0.0,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,-0.343367
1999997,670.0,0.0,6.237255,0.017694,0.000000,1.000000,0.0,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,-1.559153
1999998,490.0,0.0,3.114298,0.127890,0.000000,1.000000,0.0,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,0.837201


In [10]:
X_test

,price,discount,elast,redemption_rate,costumer_redemption_rate,discount_buy,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_WOE,product_WOE
2000000,688.0,0.0,7.511988,0.039216,NaN,NaN,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-0.541070
2000001,560.0,0.0,3.029996,0.104438,0.000000,1.000000,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,0.638078
2000002,773.0,0.0,2.149284,0.037890,0.000000,1.000000,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-0.776683
2000003,722.0,0.0,9.578316,0.020378,0.000000,1.000000,0.0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-1.401178
2000004,620.0,0.0,6.839580,0.057287,0.000000,0.272727,0.0,0,1,0,19.0,0.0,0.0,0.0,0.089888,0.039796,-0.128700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499995,549.0,0.0,3.938279,0.032201,0.666667,0.250000,0.3,0,0,1,19.0,0.0,0.0,0.0,0.123596,0.071599,-0.878618
2499996,702.0,0.3,4.741070,0.044372,0.000000,1.000000,0.0,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,-0.343367
2499997,670.0,0.0,4.759275,0.017694,0.000000,1.000000,0.3,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,-1.559153
2499998,490.0,0.0,2.050100,0.127890,0.000000,1.000000,0.3,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,0.837201


In [11]:
# FITTING THE MODEL
model.fit(X_train, y_train)
y_hat = model.predict(X_train)
log_loss_score = model.log_loss_score(y_train, y_hat)
log_loss_score

0.07608442112045971

In [12]:
y_hat = model.predict(X_test)
model.log_loss_score(y_test, y_hat)

# we have leakage i guess --> redemption features --> no leakage any more if we build redemption features on data before the train data --> not the best way

0.07692344943582866

In [14]:
coupon_creator = CouponCreator(model)

In [15]:
X_template = X_test.copy()
X_template['discount'] = None
X_template['substitue_discount'] = 0
X_template

,price,discount,elast,redemption_rate,costumer_redemption_rate,discount_buy,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_WOE,product_WOE
2000000,688.0,None,7.511988,0.039216,NaN,NaN,0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-0.541070
2000001,560.0,None,3.029996,0.104438,0.000000,1.000000,0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,0.638078
2000002,773.0,None,2.149284,0.037890,0.000000,1.000000,0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-0.776683
2000003,722.0,None,9.578316,0.020378,0.000000,1.000000,0,0,1,0,103.0,0.0,0.0,0.0,0.000000,0.039796,-1.401178
2000004,620.0,None,6.839580,0.057287,0.000000,0.272727,0,0,1,0,19.0,0.0,0.0,0.0,0.089888,0.039796,-0.128700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499995,549.0,None,3.938279,0.032201,0.666667,0.250000,0,0,0,1,19.0,0.0,0.0,0.0,0.123596,0.071599,-0.878618
2499996,702.0,None,4.741070,0.044372,0.000000,1.000000,0,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,-0.343367
2499997,670.0,None,4.759275,0.017694,0.000000,1.000000,0,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,-1.559153
2499998,490.0,None,2.050100,0.127890,0.000000,1.000000,0,0,0,1,103.0,0.0,0.0,0.0,0.000000,0.071599,0.837201


In [39]:
config['model']['discounts']

[0.15, 0.2, 0.25, 0.3]

In [16]:
revenue = coupon_creator.get_revenue(X_template, config['model']['discounts'])
revenue

,week,shopper,product,price,product_cat,discount,probabilities,exp_revenue,delta_revenue
2000000,89,0,0,688.0,0,0.0,0.001172,0.806377,None
2000001,89,0,1,560.0,0,0.0,0.001359,0.760951,None
2000002,89,0,2,773.0,0,0.0,0.001115,0.862124,None
2000003,89,0,3,722.0,0,0.0,0.001078,0.778482,None
2000004,89,0,4,620.0,0,0.0,0.097057,60.175537,None
...,...,...,...,...,...,...,...,...,...
2499995,89,1999,245,549.0,24,0.3,0.349152,134.178939,64.765404
2499996,89,1999,246,702.0,24,0.3,0.006459,3.173791,2.309909
2499997,89,1999,247,670.0,24,0.3,0.004646,2.178869,1.695355
2499998,89,1999,248,490.0,24,0.3,0.005962,2.044963,1.256793


In [17]:
coupon_creator.get_top_coupons(revenue, 5)

,shopper,week,coupon,product,discount,product_cat
2000067,0,89,0,67,0.15,6
2000171,0,89,1,171,0.15,17
2000210,0,89,2,210,0.30,21
2000157,0,89,3,157,0.15,15
2000076,0,89,4,76,0.20,7
...,...,...,...,...,...,...
2062262,249,89,0,12,0.20,1
2062339,249,89,1,89,0.20,8
2062363,249,89,2,113,0.20,11
2062475,249,89,3,225,0.20,22


In [20]:
coupons = coupon_creator.run(discounts, X_template)
coupons

,shopper,week,coupon,product,discount,product_cat
2000067,0,89,0,67,0.15,6
2000171,0,89,1,171,0.15,17
2000210,0,89,2,210,0.30,21
2000157,0,89,3,157,0.15,15
2000076,0,89,4,76,0.20,7
...,...,...,...,...,...,...
2062262,249,89,0,12,0.20,1
2062339,249,89,1,89,0.20,8
2062363,249,89,2,113,0.20,11
2062475,249,89,3,225,0.20,22


#### **Validation that two subtitutes won't receive discounts**

In [30]:
import numpy as np

In [35]:
n_unique_cats = coupons[['shopper', 'product_cat']].groupby('shopper')['product_cat'].apply(lambda x: np.unique(x).shape[0])
n_unique_cats 

shopper
0      5
1      5
2      5
3      5
4      5
      ..
245    5
246    5
247    5
248    5
249    5
Name: product_cat, Length: 250, dtype: int64

In [36]:
np.unique(n_unique_cats)

array([5])